### Merge the MTurk results with the Qualtrics results

In [1]:
import glob
import os

import pandas as pd
import numpy as np

import mtglobals

### (a) Load data downloaded from MTurk

In [2]:
os.getcwd()

'C:\\mturk-twostage\\code'

In [3]:
accepted_fpaths = glob.glob("../results_2stage/accepted_df_*.pkl")
accepted_fpaths

['../results_2stage\\accepted_df_20221014_131324.pkl',
 '../results_2stage\\accepted_df_20221014_140242.pkl',
 '../results_2stage\\accepted_df_20221015_170636.pkl']

In [4]:
#mt_df = pd.read_csv("../results/mturk_2stage/pilot_results.csv")
mt_df = pd.read_pickle(accepted_fpaths[-1])

In [5]:
mt_df.head(1)

,worker_id,wage,accepted,stage2_creation,stage2_creation_str,stage2_accept_time,stage2_submit_time,lwage,age,onlinehrs_str,reason,stage1_accept_time,stage1_submit_time,onlinehrs,sophisticated,suspended
0,A101J0P2UYUKA1,1.05,1,2022-10-13 15:32:15,2022-10-13 15:32:15-07:00,2022-10-13 15:41:19-07:00,2022-10-13 15:48:31-07:00,0.04879,23,40,skills,2022-10-13 15:01:17-07:00,2022-10-13 15:03:12-07:00,40.0,0,False


In [6]:
mt_rename = {
    'Answer.age': 'age',
    'Answer.onlinehrs': 'onlinehrs',
    'Answer.reason': 'reason',
    'worker_id': 'mt_id',
    'completed_stage2': 'submitted_hit',
}
mt_df.rename(columns=mt_rename, inplace=True)

### (b) Load Qualtrics data

In [7]:
xlsx_files = glob.glob("../results_2stage/*.xlsx")
xlsx_files

['../results_2stage\\Job+Quality+NonWM+12.0_October+14,+2022_14.43.xlsx',
 '../results_2stage\\Job+Quality+NonWM+13.0_October+15,+2022_18.07.xlsx']

In [8]:
# If only loading one file
qual_df = pd.read_excel(xlsx_files[-1])
qual_df = qual_df.iloc[1:].copy()
qual_df['qualtrics_source'] = os.path.basename(xlsx_files[-1])

C:\Users\jpjac\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [9]:
# If loading multiple
def load_multiple_xlsx():
    all_qual_dfs = []
    for cur_xlsx_file in xlsx_files:
        print(cur_xlsx_file)
        cur_qual_df = pd.read_excel(cur_xlsx_file)
        cur_qual_df = cur_qual_df.iloc[1:].copy()
        cur_qual_df['qualtrics_source'] = os.path.basename(cur_xlsx_file)
        all_qual_dfs.append(cur_qual_df)
    qual_df = pd.concat(all_qual_dfs)

In [10]:
qual_rename = {
    'Duration (in seconds)': 'duration_seconds',
}
qual_df.rename(columns=qual_rename, inplace=True)
qual_drop = ['RecipientLastName','RecipientFirstName',
             'RecipientEmail','ExternalReference','LocationLatitude','LocationLongitude',
             'Finished']
qual_df.drop(columns=qual_drop, inplace=True)

In [11]:
qual_df.drop(columns=[c for c in qual_df.columns if c.startswith("name_")], inplace=True)
qual_df.drop(columns=[c for c in qual_df.columns if c.startswith("cur_")], inplace=True)
qual_df.drop(columns=[c for c in qual_df.columns if c.startswith("val_")], inplace=True)
qual_df.drop(columns=[c for c in qual_df.columns if c.endswith("_tense")], inplace=True)

In [12]:
qual_df.sort_values(by=['mt_id','RecordedDate'], inplace=True)

In [13]:
# And keep only the most recent
qual_df_nodup = qual_df[~qual_df.duplicated(subset=['mt_id'], keep='last')].copy()

In [14]:
qual_df_nodup

,StartDate,EndDate,Status,IPAddress,Progress,duration_seconds,RecordedDate,ResponseId,DistributionChannel,UserLanguage,...,generated_jobtitle_22,generated_jobtitle_23,generated_jobtitle_24,generated_jobtitle_25,generated_jobtitle_26,generated_jobtitle_27,generated_jobtitle_28,generated_jobtitle_29,generated_jobtitle_30,qualtrics_source
348,2022-10-13 16:42:13,2022-10-13 16:48:21,IP Address,147.124.221.155,100.0,367.0,2022-10-13 16:48:22.384000,R_wYPTmiPyILfMiYx,anonymous,EN,...,INFORMATION TECHNOLOGY,INFORMATION TECHNOLOGY,INFORMATION TECHNOLOGY,INFORMATION TECHNOLOGY,INFORMATION TECHNOLOGY,INFORMATION TECHNOLOGY,INFORMATION TECHNOLOGY,INFORMATION TECHNOLOGY,INFORMATION TECHNOLOGY,"Job+Quality+NonWM+13.0_October+15,+2022_18.07...."
35,2022-10-03 23:39:50,2022-10-03 23:45:00,IP Address,208.218.230.165,100.0,310.0,2022-10-03 23:45:00.912000,R_1FyRO1zpuPvF7AI,anonymous,EN,...,Healthcare,Healthcare,Healthcare,Healthcare,Healthcare,Healthcare,Healthcare,Healthcare,Healthcare,"Job+Quality+NonWM+13.0_October+15,+2022_18.07...."
442,2022-10-13 19:10:22,2022-10-13 19:48:43,IP Address,45.196.144.33,100.0,2300.0,2022-10-13 19:48:43.694000,R_4UDbohxXH4PoNTX,anonymous,EN,...,IT,IT,IT,IT,IT,IT,IT,IT,IT,"Job+Quality+NonWM+13.0_October+15,+2022_18.07...."
595,2022-10-13 20:46:36,2022-10-13 21:04:08,IP Address,208.194.196.137,9.0,1052.0,2022-10-14 21:04:16.411000,R_1o6xtc6VU3ebaIy,anonymous,EN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Job+Quality+NonWM+13.0_October+15,+2022_18.07...."
166,2022-10-08 00:25:04,2022-10-08 00:28:12,IP Address,73.9.146.59,9.0,187.0,2022-10-09 00:28:20.360000,R_2CNa8f5fkqEf9Kj,anonymous,EN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Job+Quality+NonWM+13.0_October+15,+2022_18.07...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,2022-10-15 11:20:57,2022-10-15 11:39:35,IP Address,208.195.171.132,100.0,1117.0,2022-10-15 11:39:36.498000,R_1nO39fbXfHencu6,anonymous,EN,...,Marketing Manager,Marketing Manager,Marketing Manager,Marketing Manager,Marketing Manager,Marketing Manager,Marketing Manager,Marketing Manager,Marketing Manager,"Job+Quality+NonWM+13.0_October+15,+2022_18.07...."
38,2022-10-03 23:28:50,2022-10-03 23:47:01,IP Address,12.160.225.181,100.0,1091.0,2022-10-03 23:47:02.401000,R_2ylNADheACI4JpO,anonymous,EN,...,ADMIN,ADMIN,ADMIN,ADMIN,ADMIN,ADMIN,ADMIN,ADMIN,ADMIN,"Job+Quality+NonWM+13.0_October+15,+2022_18.07...."
185,2022-10-10 23:10:09,2022-10-10 23:31:50,IP Address,208.218.230.79,100.0,1300.0,2022-10-10 23:31:51.047000,R_1Q5CgEW0SGP6Rju,anonymous,EN,...,Middle level manager,Middle level manager,Middle level manager,Middle level manager,Middle level manager,Middle level manager,Middle level manager,Middle level manager,Middle level manager,"Job+Quality+NonWM+13.0_October+15,+2022_18.07...."
361,2022-10-13 16:54:21,2022-10-13 16:58:07,IP Address,208.216.21.76,100.0,226.0,2022-10-13 16:58:08.946000,R_1JQKJoMb00c5BT8,anonymous,EN,...,information technology,information technology,information technology,information technology,information technology,information technology,information technology,information technology,information technology,"Job+Quality+NonWM+13.0_October+15,+2022_18.07...."


In [15]:
#qual_df[qual_df.duplicated(subset=['mt_id'], keep=False)]

### And merge

In [16]:
merged_df = mt_df.merge(qual_df_nodup, how='left', on='mt_id', indicator=True)

In [17]:
# Resolve ties by sorting by date
merged_df.sort_values(by=['mt_id','RecordedDate'], inplace=True)
#merge_df['mt_id']
merged_df

,mt_id,wage_x,accepted,stage2_creation,stage2_creation_str,stage2_accept_time,stage2_submit_time,lwage,age,onlinehrs_str,...,generated_jobtitle_23,generated_jobtitle_24,generated_jobtitle_25,generated_jobtitle_26,generated_jobtitle_27,generated_jobtitle_28,generated_jobtitle_29,generated_jobtitle_30,qualtrics_source,_merge
0,A101J0P2UYUKA1,1.05,1,2022-10-13 15:32:15,2022-10-13 15:32:15-07:00,2022-10-13 15:41:19-07:00,2022-10-13 15:48:31-07:00,0.048790,23,40,...,INFORMATION TECHNOLOGY,INFORMATION TECHNOLOGY,INFORMATION TECHNOLOGY,INFORMATION TECHNOLOGY,INFORMATION TECHNOLOGY,INFORMATION TECHNOLOGY,INFORMATION TECHNOLOGY,INFORMATION TECHNOLOGY,"Job+Quality+NonWM+13.0_October+15,+2022_18.07....",both
1,A10AKR84P1WXHL,0.98,1,2022-10-03 22:26:00,2022-10-03 22:26:00-07:00,2022-10-03 22:28:30-07:00,2022-10-03 23:05:39-07:00,-0.020203,27,36,...,Healthcare,Healthcare,Healthcare,Healthcare,Healthcare,Healthcare,Healthcare,Healthcare,"Job+Quality+NonWM+13.0_October+15,+2022_18.07....",both
2,A10W3EILX06DFO,0.50,1,2022-10-13 17:29:28,2022-10-13 17:29:28-07:00,2022-10-13 18:07:12-07:00,2022-10-13 19:03:30-07:00,-0.693147,35,40,...,IT,IT,IT,IT,IT,IT,IT,IT,"Job+Quality+NonWM+13.0_October+15,+2022_18.07....",both
3,A114Q54Z648PKO,1.05,1,2022-10-13 18:37:57,2022-10-13 18:37:57-07:00,2022-10-13 19:41:48-07:00,2022-10-13 20:30:31-07:00,0.048790,25,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Job+Quality+NonWM+13.0_October+15,+2022_18.07....",both
4,A1198W1SPF1R4,1.02,0,2022-10-07 14:06:16,2022-10-07 14:06:16-07:00,NaT,NaT,0.019803,54,40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Job+Quality+NonWM+13.0_October+15,+2022_18.07....",both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,AZFXFW2ZDX54B,1.00,1,2022-10-14 18:56:27,2022-10-14 18:56:27-07:00,2022-10-15 10:20:48-07:00,2022-10-15 10:39:45-07:00,0.000000,25,120,...,Marketing Manager,Marketing Manager,Marketing Manager,Marketing Manager,Marketing Manager,Marketing Manager,Marketing Manager,Marketing Manager,"Job+Quality+NonWM+13.0_October+15,+2022_18.07....",both
796,AZH32KEBRPRES,0.98,1,2022-10-03 22:20:25,2022-10-03 22:20:25-07:00,2022-10-03 22:22:15-07:00,2022-10-03 22:47:16-07:00,-0.020203,32,25,...,ADMIN,ADMIN,ADMIN,ADMIN,ADMIN,ADMIN,ADMIN,ADMIN,"Job+Quality+NonWM+13.0_October+15,+2022_18.07....",both
797,AZNWVHY3AUSNT,0.95,1,2022-10-10 22:04:06,2022-10-10 22:04:06-07:00,2022-10-10 22:07:03-07:00,2022-10-10 22:32:03-07:00,-0.051293,51,48,...,Middle level manager,Middle level manager,Middle level manager,Middle level manager,Middle level manager,Middle level manager,Middle level manager,Middle level manager,"Job+Quality+NonWM+13.0_October+15,+2022_18.07....",both
798,AZPXDIOTEH989,0.50,1,2022-10-13 15:32:47,2022-10-13 15:32:47-07:00,2022-10-13 15:37:33-07:00,2022-10-13 15:58:15-07:00,-0.693147,25,35,...,information technology,information technology,information technology,information technology,information technology,information technology,information technology,information technology,"Job+Quality+NonWM+13.0_October+15,+2022_18.07....",both


In [18]:
merged_df.drop(columns=['wage_y'], inplace=True)
merged_df.rename(columns={'wage_x': 'wage_mturk'}, inplace=True)

In [19]:
#merge_df[merge_df.duplicated(subset=['mt_id'], keep='last')]

In [20]:
# After merging, annoyingly, we have to re-convert the numeric cols
obj_vars = ['Progress','duration_seconds']
for cur_var in obj_vars:
    merged_df[cur_var] = pd.to_numeric(merged_df[cur_var])
all_na_vars = ['race_6_TEXT','gender_4_TEXT']
for cur_var in all_na_vars:
    merged_df[cur_var] = ""

In [21]:
def gen_case(row_data):
    merge_result = row_data['_merge']
    if merge_result == 'left_only':
        return "rejected"
    # Here we know they started the survey, but not if they finished
    prog = row_data['Progress']
    if prog == 100.0:
        # They completed the Qualtrics survey
        return "finished_survey"
    return "did_not_finish"
merged_df['result'] = merged_df.apply(gen_case, axis=1)

In [22]:
merged_df['race_6_TEXT']

0       
1       
2       
3       
4       
      ..
795     
796     
797     
798     
799     
Name: race_6_TEXT, Length: 800, dtype: object

In [23]:
#with pd.option_context('max_columns', 100):
#    display(merge_df)

Due to annoying problems with to_stata, need to write to .csv then use `03d_LabelVars.ipynb` to generate the .dta with labels

In [24]:
timestamp = mtglobals.gen_timestamp()

In [25]:
csv_fpath = f"../results_2stage/pilot_results_qualtrics_{timestamp}.csv"

In [26]:
merged_df.to_csv(csv_fpath, index=False)

In [27]:
#merge_df.to_stata("../results/mturk_2stage/pilot_results.dta", variable_labels=labels)

In [28]:
merged_df.to_pickle(csv_fpath.replace(".csv",".pkl"))

In [29]:
merged_df

,mt_id,wage_mturk,accepted,stage2_creation,stage2_creation_str,stage2_accept_time,stage2_submit_time,lwage,age,onlinehrs_str,...,generated_jobtitle_24,generated_jobtitle_25,generated_jobtitle_26,generated_jobtitle_27,generated_jobtitle_28,generated_jobtitle_29,generated_jobtitle_30,qualtrics_source,_merge,result
0,A101J0P2UYUKA1,1.05,1,2022-10-13 15:32:15,2022-10-13 15:32:15-07:00,2022-10-13 15:41:19-07:00,2022-10-13 15:48:31-07:00,0.048790,23,40,...,INFORMATION TECHNOLOGY,INFORMATION TECHNOLOGY,INFORMATION TECHNOLOGY,INFORMATION TECHNOLOGY,INFORMATION TECHNOLOGY,INFORMATION TECHNOLOGY,INFORMATION TECHNOLOGY,"Job+Quality+NonWM+13.0_October+15,+2022_18.07....",both,finished_survey
1,A10AKR84P1WXHL,0.98,1,2022-10-03 22:26:00,2022-10-03 22:26:00-07:00,2022-10-03 22:28:30-07:00,2022-10-03 23:05:39-07:00,-0.020203,27,36,...,Healthcare,Healthcare,Healthcare,Healthcare,Healthcare,Healthcare,Healthcare,"Job+Quality+NonWM+13.0_October+15,+2022_18.07....",both,finished_survey
2,A10W3EILX06DFO,0.50,1,2022-10-13 17:29:28,2022-10-13 17:29:28-07:00,2022-10-13 18:07:12-07:00,2022-10-13 19:03:30-07:00,-0.693147,35,40,...,IT,IT,IT,IT,IT,IT,IT,"Job+Quality+NonWM+13.0_October+15,+2022_18.07....",both,finished_survey
3,A114Q54Z648PKO,1.05,1,2022-10-13 18:37:57,2022-10-13 18:37:57-07:00,2022-10-13 19:41:48-07:00,2022-10-13 20:30:31-07:00,0.048790,25,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Job+Quality+NonWM+13.0_October+15,+2022_18.07....",both,did_not_finish
4,A1198W1SPF1R4,1.02,0,2022-10-07 14:06:16,2022-10-07 14:06:16-07:00,NaT,NaT,0.019803,54,40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Job+Quality+NonWM+13.0_October+15,+2022_18.07....",both,did_not_finish
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,AZFXFW2ZDX54B,1.00,1,2022-10-14 18:56:27,2022-10-14 18:56:27-07:00,2022-10-15 10:20:48-07:00,2022-10-15 10:39:45-07:00,0.000000,25,120,...,Marketing Manager,Marketing Manager,Marketing Manager,Marketing Manager,Marketing Manager,Marketing Manager,Marketing Manager,"Job+Quality+NonWM+13.0_October+15,+2022_18.07....",both,finished_survey
796,AZH32KEBRPRES,0.98,1,2022-10-03 22:20:25,2022-10-03 22:20:25-07:00,2022-10-03 22:22:15-07:00,2022-10-03 22:47:16-07:00,-0.020203,32,25,...,ADMIN,ADMIN,ADMIN,ADMIN,ADMIN,ADMIN,ADMIN,"Job+Quality+NonWM+13.0_October+15,+2022_18.07....",both,finished_survey
797,AZNWVHY3AUSNT,0.95,1,2022-10-10 22:04:06,2022-10-10 22:04:06-07:00,2022-10-10 22:07:03-07:00,2022-10-10 22:32:03-07:00,-0.051293,51,48,...,Middle level manager,Middle level manager,Middle level manager,Middle level manager,Middle level manager,Middle level manager,Middle level manager,"Job+Quality+NonWM+13.0_October+15,+2022_18.07....",both,finished_survey
798,AZPXDIOTEH989,0.50,1,2022-10-13 15:32:47,2022-10-13 15:32:47-07:00,2022-10-13 15:37:33-07:00,2022-10-13 15:58:15-07:00,-0.693147,25,35,...,information technology,information technology,information technology,information technology,information technology,information technology,information technology,"Job+Quality+NonWM+13.0_October+15,+2022_18.07....",both,finished_survey
